In [40]:
import os
import pandas as pd
prot_list = pd.read_csv("protein_seqs.csv")['uniprot_id'].tolist()

In [41]:
import requests
from tqdm import tqdm
import json
folder_path = "prot_pdb"
os.makedirs(folder_path, exist_ok=True)
error_list = []
for prot in tqdm(prot_list):
    try:
        if os.path.exists(os.path.join(folder_path, f"{prot}.pdb")):
            continue
        url = f"https://alphafold.ebi.ac.uk/api/prediction/{prot}"
        request = requests.get(url)
        # print(json.dumps(request.json(), indent=2))
        pdb_url = request.json()[0]['pdbUrl']
        pdb_request = requests.get(pdb_url)
        pdb_content = pdb_request.text
        with open(os.path.join(folder_path, f"{prot}.pdb"), "w") as file:
            file.write(pdb_content)
    except Exception as e:
        error_list.append(prot)
        print(f"Error for {prot}: {e}")


 14%|█▍        | 34/238 [00:00<00:00, 303.08it/s]

Error for P15659: 0
Error for P03430: 0
Error for Q03164: 0
Error for Q86U86-5: 0


 35%|███▌      | 84/238 [00:49<02:04,  1.23it/s] 

Error for P04618: 0
Error for P25054: 0


 39%|███▉      | 94/238 [00:59<01:46,  1.35it/s]

...

 98%|█████████▊| 233/238 [03:20<00:05,  1.14s/it]

Error for P10226: 0


100%|█████████▉| 237/238 [03:22<00:00,  1.29it/s]

Error for P10227: 0


100%|██████████| 238/238 [03:24<00:00,  1.17it/s]


In [43]:
print(len(prot_list), len(os.listdir(folder_path)))
print(len(error_list))

238 219
19


In [45]:
error_list = pd.DataFrame({"uniprot_id": error_list})
error_list.to_csv("prot_error_list.csv", index=False)